In [94]:
import pandas as pd
import os
import seaborn as sns
#from skbio.diversity import alpha_diversity
#from skbio.diversity import beta_diversity
#from skbio import TreeNode
from io import StringIO
#from skbio.stats.ordination import pcoa
from skbio.stats.composition import ancom


In [95]:
path = '//gfs/data/curated_metagenomes_kraken2/'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.csv' in file:
            files.append(os.path.join(path, file))
print(files)

['//gfs/data/curated_metagenomes_kraken2/RampelliS_2015.csv', '//gfs/data/curated_metagenomes_kraken2/AsnicarF_2017.csv', '//gfs/data/curated_metagenomes_kraken2/ChngKR_2016.csv', '//gfs/data/curated_metagenomes_kraken2/FengQ_2015.csv', '//gfs/data/curated_metagenomes_kraken2/Heitz-BuschartA_2016.csv', '//gfs/data/curated_metagenomes_kraken2/KarlssonFH_2013.csv', '//gfs/data/curated_metagenomes_kraken2/LeChatelierE_2013.csv', '//gfs/data/curated_metagenomes_kraken2/LiuW_2016.csv', '//gfs/data/curated_metagenomes_kraken2/LomanNJ_2013.csv', '//gfs/data/curated_metagenomes_kraken2/Obregon-TitoAJ_2015.csv', '//gfs/data/curated_metagenomes_kraken2/QinJ_2012.csv', '//gfs/data/curated_metagenomes_kraken2/QinN_2014.csv', '//gfs/data/curated_metagenomes_kraken2/SchirmerM_2016.csv', '//gfs/data/curated_metagenomes_kraken2/VatanenT_2016.csv', '//gfs/data/curated_metagenomes_kraken2/VogtmannE_2016.csv', '//gfs/data/curated_metagenomes_kraken2/XieH_2016.csv', '//gfs/data/curated_metagenomes_kraken2

In [96]:
def get_data_for_sample(path_to_file):
    dataset = path_to_file.split('/')[-1]
    df = pd.read_csv(path_to_file, delimiter = ',')
    df['Dataset'] = dataset
    return df

In [97]:
df = get_data_for_sample(files[10])

In [ ]:
get_table_and_grouping(df, column_to_group='STUDY_CONDITION', rank='F'):
    df_2 = df.loc[df['Rank'] == rank]
    df_2 = df_2[['Sample_ID', 'Name', 'Abundance', column_to_group]]
    # Handleing zero values: pseudo-count = 1
    df_2["Abundance"] = [1 if ab == 0 else ab for ab in df_2["Abundance"]]
    
    table = df_2.pivot(index='Sample_ID', columns='Name', values='Abundance')
    
    # Dropping sparse columns
    counts = {}
    to_drop = []
    for column in table.columns:
        counts[column] = table[column].loc[table[column].notnull()].count()
        if counts[column] < 20:
            to_drop.append(column)
    table = table.drop(columns = to_drop)
    
    # Handleing remaining Nan values
    table = table.fillna(1)
    
    grouping = df_2[['Sample_ID', column_to_group]].drop_duplicates()
    grouping.index = grouping["Sample_ID"]
    grouping = grouping.drop(columns = ["Sample_ID"])
    
    grouping_ser = grouping[column_to_group]
    
    results = ancom(table, grouping_ser)
    return results

In [98]:
df.head()

,Unnamed: 0,Sample_ID,Dataset,Rank,Name,Abundance,STUDY_CONDITION,BODY_SITE,SUBJECTID,COUNTRY,DISEASE,AGE,AGE_CATEGORY
0,0,SRR341631,QinJ_2012.csv,U,unclassified,90.97,control,stool,NLM016,CHN,none,23.0,adult
1,1,SRR341631,QinJ_2012.csv,R,root,9.03,control,stool,NLM016,CHN,none,23.0,adult
2,2,SRR341631,QinJ_2012.csv,R1,cellular organisms,9.00,control,stool,NLM016,CHN,none,23.0,adult
3,3,SRR341631,QinJ_2012.csv,D,Bacteria,8.95,control,stool,NLM016,CHN,none,23.0,adult
4,4,SRR341631,QinJ_2012.csv,D1,FCB group,5.24,control,stool,NLM016,CHN,none,23.0,adult


In [99]:
df['STUDY_CONDITION'].drop_duplicates()

0          control
5674           T2D
1231085        NaN
Name: STUDY_CONDITION, dtype: object

In [100]:
df_2 = df.loc[df['Rank'] == 'F']
df_2 = df_2[['Sample_ID', 'Name', 'Abundance', 'STUDY_CONDITION']]

In [101]:
df_2.head()

,Sample_ID,Name,Abundance,STUDY_CONDITION
9,SRR341631,Bacteroidaceae,2.33,control
34,SRR341631,Prevotellaceae,1.65,control
53,SRR341631,Tannerellaceae,0.31,control
64,SRR341631,Odoribacteraceae,0.09,control
68,SRR341631,Rikenellaceae,0.07,control


In [102]:
df_2["Abundance"] = [1 if ab == 0 else ab for ab in df_2["Abundance"]]

In [103]:
df_2.head()

,Sample_ID,Name,Abundance,STUDY_CONDITION
9,SRR341631,Bacteroidaceae,2.33,control
34,SRR341631,Prevotellaceae,1.65,control
53,SRR341631,Tannerellaceae,0.31,control
64,SRR341631,Odoribacteraceae,0.09,control
68,SRR341631,Rikenellaceae,0.07,control


In [104]:
table = df_2.pivot(index='Sample_ID', columns='Name', values='Abundance')

In [105]:
table.head()

Name,Hominidae,Sphaerobacteraceae,Amoebophilaceae,Anaeromyxobacteraceae,Aphanothecaceae,Archangiaceae,Bacteroidaceae,Barnesiellaceae,Bernardetiaceae,Blattabacteriaceae,...,Polydnaviridae,Polyomaviridae,Poxviridae,Reoviridae,Retroviridae,Sphaerolipoviridae,Tectiviridae,Tolecusatellitidae,Totiviridae,Tristromaviridae
Sample_ID,,,,,,,,,,,,,,,,,,,,,
SRR1778450,0.01,NaN,NaN,1.0,1.0,NaN,0.02,1.00,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR341581,0.01,NaN,1.0,1.0,1.0,1.0,55.78,0.01,1.0,1.0,...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR341582,0.02,1.0,1.0,1.0,1.0,1.0,31.29,0.01,1.0,1.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
SRR341583,0.05,1.0,1.0,1.0,1.0,1.0,3.67,0.02,1.0,1.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR341584,0.02,1.0,1.0,1.0,1.0,1.0,14.80,0.02,1.0,1.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
counts = {}
to_drop = []
for column in table.columns:
    counts[column] = table[column].loc[table[column].notnull()].count()
    if counts[column] < 20:
        to_drop.append(column)

In [107]:
table = table.drop(columns = to_drop)

In [108]:
table = table.fillna(1)

In [109]:
table.head()

Name,Hominidae,Sphaerobacteraceae,Amoebophilaceae,Anaeromyxobacteraceae,Aphanothecaceae,Archangiaceae,Bacteroidaceae,Barnesiellaceae,Bernardetiaceae,Blattabacteriaceae,...,Microviridae,Mimiviridae,Nudiviridae,Papillomaviridae,Phycodnaviridae,Polydnaviridae,Polyomaviridae,Poxviridae,Reoviridae,Totiviridae
Sample_ID,,,,,,,,,,,,,,,,,,,,,
SRR1778450,0.01,1.0,1.0,1.0,1.0,1.0,0.02,1.00,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
SRR341581,0.01,1.0,1.0,1.0,1.0,1.0,55.78,0.01,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
SRR341582,0.02,1.0,1.0,1.0,1.0,1.0,31.29,0.01,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
SRR341583,0.05,1.0,1.0,1.0,1.0,1.0,3.67,0.02,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
SRR341584,0.02,1.0,1.0,1.0,1.0,1.0,14.80,0.02,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [110]:
grouping = df_2[['Sample_ID', 'STUDY_CONDITION']].drop_duplicates()

In [111]:
grouping.head()

,Sample_ID,STUDY_CONDITION
9,SRR341631,control
5683,SRR341611,T2D
12101,SRR341592,T2D
17587,SRR341650,control
24219,SRR341619,control


In [112]:
grouping.index = grouping["Sample_ID"]

In [113]:
grouping = grouping.drop(columns = ["Sample_ID"])

In [114]:
grouping.head()

,STUDY_CONDITION
Sample_ID,
SRR341631,control
SRR341611,T2D
SRR341592,T2D
SRR341650,control
SRR341619,control


In [115]:
grouping_ser = grouping['STUDY_CONDITION']

In [116]:
results = ancom(table, grouping_ser)

ValueError: Cannot handle missing values in `grouping`.

In [75]:
result_df = results[0]
percentile_df = results[1]

In [76]:
result_df.head()

,W,Reject null hypothesis
Name,,
Hominidae,34,False
Sphaerobacteraceae,24,False
Amoebophilaceae,24,False
Anaeromyxobacteraceae,24,False
Aphanothecaceae,24,False


In [81]:
list(result_df.loc[result_df["Reject null hypothesis"]].index)

['                Chlorobiaceae',
 '                Prevotellaceae',
 '              Bifidobacteriaceae',
 '            Spirochaetaceae']

In [85]:
percentile_df[50.0].loc['                Chlorobiaceae']

Group
ITA    1.00
TZA    0.01
Name:                 Chlorobiaceae, dtype: float64

In [86]:
percentile_df[50.0].loc['                Prevotellaceae']

Group
ITA    0.11
TZA    0.74
Name:                 Prevotellaceae, dtype: float64

In [87]:
percentile_df[50.0].loc['              Bifidobacteriaceae']

Group
ITA    1.98
TZA    0.02
Name:               Bifidobacteriaceae, dtype: float64

In [88]:
percentile_df[50.0].loc['            Spirochaetaceae']

Group
ITA    0.01
TZA    0.29
Name:             Spirochaetaceae, dtype: float64